# Impact Detection and Player Tracking

This notebook allows you to:
1. Load a video and specify impact timeframes
2. Track players using YOLOv5 + ByteTrack
3. Extract centered clips for each player
4. Automatically sort clips into "impact" and "normal" folders based on timing

## Configuration - Set Your Paths Here

In [ ]:
# CONFIGURATION
from pathlib import Path
import shutil

# Root (parent of notebooks/)
PROJECT_ROOT = Path.cwd().parent

# Set these
VIDEO_PATH = PROJECT_ROOT / "data" / "videos" / "your_video.mp4"      # ← change filename
OUTPUT_DIR = PROJECT_ROOT / "outputs" / "run_1"                       # output folder
WEIGHTS_PATH = PROJECT_ROOT / "weights" / "yolov5m-1280-football-players.pt"  # ← change if needed

video_path = VIDEO_PATH
output_base_dir = OUTPUT_DIR

if output_base_dir.exists():
    print(f"🗑️ Cleaning existing output directory: {output_base_dir}")
    shutil.rmtree(output_base_dir)

(output_base_dir / "impact").mkdir(parents=True, exist_ok=True)
(output_base_dir / "normal").mkdir(parents=True, exist_ok=True)

impact_dir = output_base_dir / "impact"
normal_dir = output_base_dir / "normal"

print(f"📁 Output directory prepared: {output_base_dir}")
print(f"📹 Video path: {video_path}")
print(f"   Exists: {'✅' if video_path.exists() else '❌'}")
print(f"🧪 Weights path: {WEIGHTS_PATH}")
print(f"   Exists: {'✅' if WEIGHTS_PATH.exists() else '❌'}")

📁 Output directory prepared: Mitch Owens
📹 Video path: Concussions\Mitch Owens - R10 STK\R10 STK vs GWS 23.14 Mitch_Owens R10 STK vs GWS Rnd10_GWS_v_STK_Q2_BCAST.mp4.mp4
   Exists: ✅


## Setup and Imports

In [2]:
import sys
sys.path.append("ByteTrack")
from yolox.tracker.byte_tracker import BYTETracker, STrack
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass
from typing import List, Optional, Tuple
import datetime
import gc

## Video Loading and Impact Time Input

In [3]:
def get_video_info(video_path):
    """Get video information without loading all frames"""
    video = cv2.VideoCapture(str(video_path))
    if not video.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
        
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video.release()
    
    return {
        'fps': fps,
        'total_frames': total_frames,
        'duration': duration,
        'width': width,
        'height': height
    }

def time_to_frame(time_str, fps):
    """Convert time string (MM:SS or SS.SS) to frame number"""
    try:
        if ':' in time_str:
            parts = time_str.split(':')
            minutes = int(parts[0])
            seconds = float(parts[1])
            total_seconds = minutes * 60 + seconds
        else:
            total_seconds = float(time_str)
        return int(total_seconds * fps)
    except:
        return None

def get_impact_timeframes():
    """Get impact timeframes from user input"""
    impact_periods = []
    
    print("\n🎯 IMPACT DETECTION SETUP")
    print("Enter impact timeframes. You can specify multiple impact periods.")
    print("Time formats supported: MM:SS (e.g., 1:30) or SS.SS (e.g., 90.5)")
    print("Press Enter without input to finish.\n")
    
    while True:
        start_time = input(f"Impact {len(impact_periods)+1} - Start time (or Enter to finish): ").strip()
        if not start_time:
            break
            
        end_time = input(f"Impact {len(impact_periods)+1} - End time: ").strip()
        if not end_time:
            print("❌ End time required. Skipping this impact period.")
            continue
        
        impact_periods.append((start_time, end_time))
        print(f"✅ Added impact period: {start_time} to {end_time}")
    
    return impact_periods

In [4]:
if not video_path.exists():
    print(f"❌ Video file not found: {video_path}")
    raise FileNotFoundError(f"Video not found: {video_path}")

# Get video information
video_info = get_video_info(video_path)
print(f"\n📊 VIDEO INFORMATION:")
print(f"   Path: {video_path}")
print(f"   Duration: {video_info['duration']:.2f} seconds")
print(f"   FPS: {video_info['fps']:.2f}")
print(f"   Resolution: {video_info['width']}x{video_info['height']}")
print(f"   Total frames: {video_info['total_frames']}")

# Get impact timeframes
impact_periods = get_impact_timeframes()

# Convert to frame numbers
impact_frames = []
for start_time, end_time in impact_periods:
    start_frame = time_to_frame(start_time, video_info['fps'])
    end_frame = time_to_frame(end_time, video_info['fps'])
    
    if start_frame is not None and end_frame is not None:
        impact_frames.append((start_frame, end_frame))
        print(f"📍 Impact: frames {start_frame}-{end_frame} ({start_time} to {end_time})")
    else:
        print(f"❌ Invalid time format: {start_time} to {end_time}")

print(f"\n✅ Setup complete! {len(impact_frames)} impact period(s) defined.")


📊 VIDEO INFORMATION:
   Path: Concussions\Mitch Owens - R10 STK\R10 STK vs GWS 23.14 Mitch_Owens R10 STK vs GWS Rnd10_GWS_v_STK_Q2_BCAST.mp4.mp4
   Duration: 69.12 seconds
   FPS: 24.99
   Resolution: 1920x1080
   Total frames: 1727

🎯 IMPACT DETECTION SETUP
Enter impact timeframes. You can specify multiple impact periods.
Time formats supported: MM:SS (e.g., 1:30) or SS.SS (e.g., 90.5)
Press Enter without input to finish.

✅ Added impact period: 0:29 to 0:31
📍 Impact: frames 724-774 (0:29 to 0:31)

✅ Setup complete! 1 impact period(s) defined.


## Model Setup

In [5]:
# Check hardware setup
print("🔧 HARDWARE SETUP:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# Load YOLOv5 model
device = '0' if torch.cuda.is_available() else 'cpu'
path_weights = "weights/yolov5m-1280-football-players.pt"

print(f"\n🤖 LOADING MODEL:")
print(f"   Weights: {path_weights}")
print(f"   Device: {device}")

yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path_weights, device=device, force_reload=True)
print("✅ Model loaded successfully!")

🔧 HARDWARE SETUP:
   PyTorch version: 2.7.0+cu128
   CUDA available: True
   GPU: NVIDIA GeForce RTX 3080 Laptop GPU

🤖 LOADING MODEL:
   Weights: weights/yolov5m-1280-football-players.pt
   Device: 0
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\ACER/.cache\torch\hub\master.zip
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING requirements:  Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.


YOLOv5  2025-10-22 Python-3.10.17 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 212 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


✅ Model loaded successfully!


In [6]:
# Class mappings and colors
ind_to_cls = {
    1: "goalkeeper",
    2: "player", 
    3: "referee"
}

colors = {
    "player": (255, 0, 0),
    "goalkeeper": (255, 0, 255),
    "referee": (0, 0, 255),
    "unknown": (128, 128, 128)
}

@dataclass
class Detection:
    xywh: List[float]
    xyxy: List[float]
    class_id: int
    class_name: str
    confidence: float
    tracker_id: Optional[int] = None

    @classmethod
    def from_results(cls, pred):
        result = []
        for x_min, y_min, x_max, y_max, confidence, class_id in pred:
            class_id = int(class_id)
            class_name = ind_to_cls.get(class_id, "unknown")
            result.append(Detection(
                xywh=[float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)],
                xyxy=[float(x_min), float(y_min), float(x_max), float(y_max)],
                class_id=class_id,
                class_name=class_name,
                confidence=float(confidence)
            ))
        return result

## ByteTracker Setup

In [7]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

def format_predictions(predictions, with_conf: bool = True):
    """Format yolo detection to ByteTrack format: (x1, y1, x2, y2, conf)"""
    frame_detections = []
    for pred in predictions:
        bbox = pred.xyxy
        conf = pred.confidence
        if with_conf:
            detection = bbox + [conf]
        else:
            detection = bbox
        frame_detections.append(detection)
    return np.array(frame_detections, dtype=float) if frame_detections else np.empty((0, 5), dtype=float)

def match_detections_with_tracks(detections, tracks):
    """Match detections with tracks using IoU"""
    if len(tracks) == 0 or len(detections) == 0:
        return detections
    
    detections_bboxes = format_predictions(detections, with_conf=False)
    tracks_bboxes = np.array([track.tlbr for track in tracks], dtype=float)
    iou = box_iou_batch(tracks_bboxes, detections_bboxes)
    track2detection = np.argmax(iou, axis=1)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            detections[detection_index].tracker_id = tracks[tracker_index].track_id
    return detections

# Initialize tracker
byte_tracker = BYTETracker(BYTETrackerArgs)
print("✅ ByteTracker initialized!")

✅ ByteTracker initialized!


## Impact Detection Functions

In [8]:
def is_impact_frame(frame_idx, impact_frames):
    """Check if a frame is within any impact period"""
    for start_frame, end_frame in impact_frames:
        if start_frame <= frame_idx <= end_frame:
            return True
    return False

def get_frame_label(frame_idx, impact_frames):
    """Get label for frame: 'impact' or 'normal'"""
    return 'impact' if is_impact_frame(frame_idx, impact_frames) else 'normal'

def analyze_clip_overlap(clip_start_frame, clip_end_frame, impact_frames, overlap_threshold=0.5):
    """
    Analyze if a clip overlaps significantly with impact periods
    Returns: 'impact' if overlap > threshold, 'normal' otherwise
    """
    clip_length = clip_end_frame - clip_start_frame
    
    # Handle zero-length clips
    if clip_length <= 0:
        # For single frame or invalid clips, check if the frame is in any impact period
        for start_frame, end_frame in impact_frames:
            if start_frame <= clip_start_frame <= end_frame:
                return 'impact'
        return 'normal'
    
    impact_overlap = 0
    
    for start_frame, end_frame in impact_frames:
        overlap_start = max(clip_start_frame, start_frame)
        overlap_end = min(clip_end_frame, end_frame)
        
        if overlap_start < overlap_end:
            impact_overlap += overlap_end - overlap_start
    
    overlap_ratio = impact_overlap / clip_length
    return 'impact' if overlap_ratio >= overlap_threshold else 'normal'

## Player Tracking and Data Collection

In [9]:
def track_players_lightweight(video_path, impact_frames, output_video=False):
    """Track players and collect data - NO FRAME STORAGE"""
    
    player_tracking_data = {}
    impact_stats = {'impact_frames': 0, 'normal_frames': 0}
    
    video = cv2.VideoCapture(str(video_path))
    if not video.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
        
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Setup video writer if needed
    video_writer = None
    if output_video:
        output_path = output_base_dir / f"tracking_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        video_writer = cv2.VideoWriter(
            str(output_path),
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (width, height)
        )
    
    try:
        frame_idx = 0
        with tqdm(total=total_frames, desc="🎯 Tracking players") as pbar:
            while True:
                ret, frame = video.read()
                if not ret:
                    break
                
                # Get frame label
                frame_label = get_frame_label(frame_idx, impact_frames)
                impact_stats[f'{frame_label}_frames'] += 1
                
                # YOLO detection
                detections = yolo_model(frame).pred[0].cpu().numpy()
                detections = Detection.from_results(detections)
                
                # Prepare detections for tracking
                detections_with_tracker = []
                for detection in detections:
                    detection.tracker_id = ""
                    detections_with_tracker.append(detection)
                
                # ByteTrack update
                tracks = byte_tracker.update(
                    output_results=format_predictions(detections_with_tracker, with_conf=True),
                    img_info=frame.shape,
                    img_size=frame.shape
                )
                
                # Match detections with tracks
                detections_with_tracker = match_detections_with_tracks(detections_with_tracker, tracks)
                
                # Collect tracking data
                for detection in detections_with_tracker:
                    if detection.tracker_id and detection.class_name in ["player", "goalkeeper", "referee"]:
                        if detection.tracker_id not in player_tracking_data:
                            player_tracking_data[detection.tracker_id] = []
                        
                        player_tracking_data[detection.tracker_id].append({
                            'frame_idx': frame_idx,
                            'bbox': detection.xyxy,
                            'class_name': detection.class_name,
                            'confidence': detection.confidence,
                            'impact_label': frame_label
                        })
                
                # Write annotated video if needed
                if output_video and video_writer:
                    # Add impact indicator
                    label_color = (0, 0, 255) if frame_label == 'impact' else (0, 255, 0)
                    label_text = "IMPACT" if frame_label == 'impact' else "NORMAL"
                    cv2.putText(frame, label_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, label_color, 3)
                    
                    # Draw detections
                    for pred in detections_with_tracker:
                        if pred.class_name != "unknown" and pred.tracker_id:
                            bbox = pred.xyxy
                            color = colors[pred.class_name]
                            cv2.rectangle(frame, tuple([int(b) for b in bbox[:2]]), 
                                        tuple([int(b) for b in bbox[2:]]), color, 3)
                            cv2.putText(frame, str(pred.tracker_id), 
                                      (int(bbox[0]), int(bbox[1])-10), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 3)
                    
                    video_writer.write(frame)
                
                frame_idx += 1
                pbar.update(1)
    
    finally:
        video.release()
        if video_writer:
            video_writer.release()
            print(f"📹 Annotated video saved: {output_path}")
    
    print(f"\n📊 TRACKING SUMMARY:")
    print(f"   Total players tracked: {len(player_tracking_data)}")
    print(f"   Impact frames: {impact_stats['impact_frames']}")
    print(f"   Normal frames: {impact_stats['normal_frames']}")
    print(f"   Impact ratio: {impact_stats['impact_frames']/total_frames*100:.1f}%")
    
    return player_tracking_data, fps

In [10]:
print("🚀 Starting player tracking...")

player_tracking_data, fps = track_players_lightweight(
    video_path, 
    impact_frames, 
    output_video=True
)

print("\n👥 PLAYER SUMMARY:")
for player_id, track_data in player_tracking_data.items():
    class_name = track_data[0]['class_name']
    impact_frames_count = sum(1 for d in track_data if d['impact_label'] == 'impact')
    normal_frames_count = len(track_data) - impact_frames_count
    
    print(f"   Player {player_id} ({class_name}): {len(track_data)} frames")
    print(f"     - Impact: {impact_frames_count} frames")
    print(f"     - Normal: {normal_frames_count} frames")

print("\n✅ Tracking completed!")

# Force cleanup
gc.collect()

🚀 Starting player tracking...


🎯 Tracking players:   0%|          | 0/1727 [00:00<?, ?it/s]C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
🎯 Tracking players:   0%|          | 1/1727 [00:00<05:11,  5.54it/s]C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocas

📹 Annotated video saved: Mitch Owens\tracking_20251022_140337.avi

📊 TRACKING SUMMARY:
   Total players tracked: 427
   Impact frames: 51
   Normal frames: 1676
   Impact ratio: 3.0%

👥 PLAYER SUMMARY:
   Player 1 (player): 113 frames
     - Impact: 0 frames
     - Normal: 113 frames
   Player 2 (player): 113 frames
     - Impact: 0 frames
     - Normal: 113 frames
   Player 3 (player): 113 frames
     - Impact: 0 frames
     - Normal: 113 frames
   Player 4 (player): 110 frames
     - Impact: 0 frames
     - Normal: 110 frames
   Player 5 (player): 113 frames
     - Impact: 0 frames
     - Normal: 113 frames
   Player 6 (player): 31 frames
     - Impact: 0 frames
     - Normal: 31 frames
   Player 7 (player): 58 frames
     - Impact: 0 frames
     - Normal: 58 frames
   Player 8 (player): 22 frames
     - Impact: 0 frames
     - Normal: 22 frames
   Player 9 (player): 113 frames
     - Impact: 0 frames
     - Normal: 113 frames
   Player 10 (player): 16 frames
     - Impact: 0 frames


3066

## Clip Extraction with Impact Classification

In [11]:
def create_player_clip(video_path, clip_frames_data, crop_size, fps, output_path):
    """Create a player clip by reloading video for specific frames"""
    
    # Validate inputs
    if not clip_frames_data:
        return False
    
    # Get frame indices we need
    frame_indices = [data['frame_idx'] for data in clip_frames_data]
    min_frame = min(frame_indices)
    max_frame = max(frame_indices)
    
    # Open video
    video = cv2.VideoCapture(str(video_path))
    if not video.isOpened():
        print(f"❌ Could not open video for clip creation")
        return False
    
    # Try different codecs
    codecs_to_try = [
        ('mp4v', '.mp4'),
        ('XVID', '.avi'), 
        ('MJPG', '.avi')
    ]
    
    clip_writer = None
    final_output_path = output_path
    
    for codec, ext in codecs_to_try:
        if not str(output_path).endswith(ext):
            final_output_path = output_path.with_suffix(ext)
        
        try:
            clip_writer = cv2.VideoWriter(
                str(final_output_path),
                cv2.VideoWriter_fourcc(*codec),
                fps,
                crop_size
            )
            
            if clip_writer.isOpened():
                break
            else:
                clip_writer.release()
        except:
            if clip_writer:
                clip_writer.release()
            continue
    
    if not clip_writer or not clip_writer.isOpened():
        video.release()
        return False
    
    try:
        frames_written = 0
        
        # Create frame data lookup
        frame_data_dict = {data['frame_idx']: data for data in clip_frames_data}
        
        # Seek to start position
        video.set(cv2.CAP_PROP_POS_FRAMES, min_frame)
        current_frame_idx = min_frame
        
        while current_frame_idx <= max_frame:
            ret, frame = video.read()
            if not ret:
                break
            
            # Process frame if it's in our list
            if current_frame_idx in frame_data_dict:
                frame_data = frame_data_dict[current_frame_idx]
                
                # Create centered crop
                bbox = frame_data['bbox']
                x1, y1, x2, y2 = bbox
                center_x, center_y = int((x1 + x2) / 2), int((y1 + y2) / 2)
                
                crop_width, crop_height = crop_size
                
                # Calculate crop boundaries with safety checks
                frame_h, frame_w = frame.shape[:2]
                
                crop_x1 = max(0, center_x - crop_width // 2)
                crop_y1 = max(0, center_y - crop_height // 2)
                crop_x2 = min(frame_w, crop_x1 + crop_width)
                crop_y2 = min(frame_h, crop_y1 + crop_height)
                
                # Adjust if boundaries are invalid
                if crop_x2 - crop_x1 < crop_width:
                    crop_x1 = max(0, crop_x2 - crop_width)
                if crop_y2 - crop_y1 < crop_height:
                    crop_y1 = max(0, crop_y2 - crop_height)
                
                # Final boundary check
                crop_x1 = max(0, min(crop_x1, frame_w - 1))
                crop_y1 = max(0, min(crop_y1, frame_h - 1))
                crop_x2 = max(crop_x1 + 1, min(crop_x2, frame_w))
                crop_y2 = max(crop_y1 + 1, min(crop_y2, frame_h))
                
                # Extract and process crop
                try:
                    cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]
                    
                    if cropped_frame.size > 0:
                        # Resize to exact dimensions
                        if cropped_frame.shape[:2] != (crop_height, crop_width):
                            cropped_frame = cv2.resize(cropped_frame, crop_size)
                        
                        # Write frame
                        clip_writer.write(cropped_frame)
                        frames_written += 1
                except:
                    pass  # Skip problematic frames
            
            current_frame_idx += 1
        
    except Exception as e:
        print(f"❌ Error in clip creation: {e}")
        return False
    
    finally:
        video.release()
        clip_writer.release()
    
    # Check if clip was created successfully
    if frames_written == 0:
        if final_output_path.exists():
            final_output_path.unlink()
        return False
    
    return True

def calculate_optimal_crop_size(track_data, player_frame_percentage=0.5, min_clip_size=(224, 224)):
    """Calculate optimal crop size from bounding box data"""
    
    total_width = sum(bbox[2] - bbox[0] for data in track_data for bbox in [data['bbox']])
    total_height = sum(bbox[3] - bbox[1] for data in track_data for bbox in [data['bbox']])
    
    avg_bbox_width = total_width / len(track_data)
    avg_bbox_height = total_height / len(track_data)
    
    crop_width = max(int(avg_bbox_width / player_frame_percentage), min_clip_size[0])
    crop_height = max(int(avg_bbox_height / player_frame_percentage), min_clip_size[1])
    
    # Make dimensions even
    crop_width = crop_width + (crop_width % 2)
    crop_height = crop_height + (crop_height % 2)
    
    return (crop_width, crop_height)

In [12]:
def extract_all_clips(player_tracking_data, video_path, fps, impact_frames, 
                      clip_duration=5.0, overlap_threshold=0.2, min_duration=1.0):
    """Extract all player clips with impact classification"""
    
    frames_per_clip = int(fps * clip_duration)
    min_frames = int(fps * min_duration)
    clip_stats = {'impact': 0, 'normal': 0, 'skipped': 0, 'failed': 0}
    
    print(f"🎬 EXTRACTING CLIPS")
    print(f"   Clip duration: {clip_duration}s ({frames_per_clip} frames)")
    print(f"   Min duration: {min_duration}s ({min_frames} frames)")
    print(f"   Output directories: {impact_dir} | {normal_dir}")
    print(f"   Overlap threshold: {overlap_threshold*100}%\n")
    
    # Calculate total clips including short players
    total_clips = 0
    for track_data in player_tracking_data.values():
        if len(track_data) < min_frames:
            continue  # Skip players below minimum duration
        elif len(track_data) < frames_per_clip:
            total_clips += 1  # One clip for the entire duration
        else:
            total_clips += max(0, len(track_data) - frames_per_clip + 1) // (frames_per_clip // 2)
    
    with tqdm(total=total_clips, desc="Creating clips") as pbar:
        for player_id, track_data in player_tracking_data.items():
            player_class = track_data[0]['class_name']
            
            # Skip players below minimum duration threshold
            if len(track_data) < min_frames:
                print(f"   ⏭️  Player {player_id}: {len(track_data)} frames ({len(track_data)/fps:.1f}s) - below min duration ({min_duration}s), skipping")
                clip_stats['skipped'] += 1
                continue
            
            crop_size = calculate_optimal_crop_size(track_data)
            
            # Handle players with less than clip_duration seconds but above min_duration
            if len(track_data) < frames_per_clip:
                print(f"   📏 Player {player_id}: {len(track_data)} frames ({len(track_data)/fps:.1f}s) - using entire duration")
                
                # Use all available frames for this player
                clip_frames_data = track_data
                
                # Determine clip classification
                clip_start_frame = clip_frames_data[0]['frame_idx']
                clip_end_frame = clip_frames_data[-1]['frame_idx']
                
                clip_label = analyze_clip_overlap(clip_start_frame, clip_end_frame, impact_frames, overlap_threshold)
                
                # Choose output directory
                output_dir = impact_dir if clip_label == 'impact' else normal_dir
                
                # Create clip filename with duration info
                duration_str = f"{len(track_data)/fps:.1f}s"
                clip_filename = f"{player_class}_{player_id}_full_{duration_str}.mp4"
                clip_path = output_dir / clip_filename
                
                # Create the clip
                success = create_player_clip(video_path, clip_frames_data, crop_size, fps, clip_path)
                
                if success:
                    clip_stats[clip_label] += 1
                else:
                    clip_stats['failed'] += 1
                
                pbar.update(1)
            
            else:
                # Handle players with enough frames for standard clips
                player_clip_count = 0
                for start_idx in range(0, len(track_data) - frames_per_clip + 1, frames_per_clip // 2):
                    clip_frames_data = track_data[start_idx:start_idx + frames_per_clip]
                    
                    # Determine clip classification
                    clip_start_frame = clip_frames_data[0]['frame_idx']
                    clip_end_frame = clip_frames_data[-1]['frame_idx']
                    
                    clip_label = analyze_clip_overlap(clip_start_frame, clip_end_frame, impact_frames, overlap_threshold)
                    
                    # Choose output directory
                    output_dir = impact_dir if clip_label == 'impact' else normal_dir
                    
                    # Create clip filename
                    clip_filename = f"{player_class}_{player_id}_clip_{player_clip_count:03d}.mp4"
                    clip_path = output_dir / clip_filename
                    
                    # Create the clip
                    success = create_player_clip(video_path, clip_frames_data, crop_size, fps, clip_path)
                    
                    if success:
                        clip_stats[clip_label] += 1
                    else:
                        clip_stats['failed'] += 1
                    
                    player_clip_count += 1
                    pbar.update(1)
    
    return clip_stats

# Clean output directories before extraction
if impact_dir.exists():
    shutil.rmtree(impact_dir)
if normal_dir.exists():
    shutil.rmtree(normal_dir)
impact_dir.mkdir(parents=True, exist_ok=True)
normal_dir.mkdir(parents=True, exist_ok=True)

# Run clip extraction
if player_tracking_data:
    print("🎬 Starting clip extraction...")
    print("🧹 Cleaned output directories")
    
    clip_stats = extract_all_clips(
        player_tracking_data=player_tracking_data,
        video_path=video_path,
        fps=fps,
        impact_frames=impact_frames,
        clip_duration=10.0,
        overlap_threshold=0.2,
        min_duration=0.5  # ← New parameter: minimum 0.5 seconds
    )
    
    print(f"\n📊 CLIP EXTRACTION SUMMARY:")
    print(f"   Impact clips: {clip_stats['impact']}")
    print(f"   Normal clips: {clip_stats['normal']}")
    print(f"   Failed clips: {clip_stats['failed']}")
    print(f"   Players skipped: {clip_stats['skipped']}")
    print(f"   Total successful: {clip_stats['impact'] + clip_stats['normal']}")
    
    print(f"\n📁 OUTPUT STRUCTURE:")
    print(f"   {output_base_dir}/")
    print(f"   ├── impact/     ({len(list(impact_dir.glob('*.mp4')))} clips)")
    print(f"   └── normal/     ({len(list(normal_dir.glob('*.mp4')))} clips)")
    
    print("\n✅ Clip extraction completed!")
else:
    print("❌ No tracking data available!")

🎬 Starting clip extraction...
🧹 Cleaned output directories
🎬 EXTRACTING CLIPS
   Clip duration: 10.0s (249 frames)
   Min duration: 0.5s (12 frames)
   Output directories: Mitch Owens\impact | Mitch Owens\normal
   Overlap threshold: 20.0%



Creating clips:   0%|          | 0/198 [00:00<?, ?it/s]

   📏 Player 1: 113 frames (4.5s) - using entire duration


Creating clips:   1%|          | 1/198 [00:00<02:06,  1.55it/s]

   📏 Player 2: 113 frames (4.5s) - using entire duration


...

   📏 Player 1429: 16 frames (0.6s) - using entire duration
   ⏭️  Player 1432: 1 frames (0.0s) - below min duration (0.5s), skipping
   📏 Player 1438: 46 frames (1.8s) - using entire duration


Creating clips: 100%|██████████| 198/198 [01:23<00:00,  4.11it/s]

   ⏭️  Player 1441: 1 frames (0.0s) - below min duration (0.5s), skipping
   ⏭️  Player 1443: 6 frames (0.2s) - below min duration (0.5s), skipping
   ⏭️  Player 1449: 3 frames (0.1s) - below min duration (0.5s), skipping
   ⏭️  Player 1450: 5 frames (0.2s) - below min duration (0.5s), skipping
   ⏭️  Player 1451: 2 frames (0.1s) - below min duration (0.5s), skipping
   📏 Player 1454: 18 frames (0.7s) - using entire duration


Creating clips: 100%|██████████| 198/198 [01:23<00:00,  2.36it/s]

   ⏭️  Player 1457: 5 frames (0.2s) - below min duration (0.5s), skipping
   ⏭️  Player 1459: 8 frames (0.3s) - below min duration (0.5s), skipping
   ⏭️  Player 1460: 2 frames (0.1s) - below min duration (0.5s), skipping

📊 CLIP EXTRACTION SUMMARY:
   Impact clips: 21
   Normal clips: 177
   Failed clips: 0
   Players skipped: 229
   Total successful: 198

📁 OUTPUT STRUCTURE:
   Mitch Owens/
   ├── impact/     (21 clips)
   └── normal/     (177 clips)

✅ Clip extraction completed!


## Summary and Analysis

In [ ]:
# Memory cleanup
gc.collect()


📋 FINAL SUMMARY

📹 VIDEO PROCESSING:
   Video: R10 STK vs GWS 23.14 Mitch_Owens R10 STK vs GWS Rnd10_GWS_v_STK_Q2_BCAST.mp4.mp4
   Duration: 69.12 seconds
   FPS: 24.99

⚡ IMPACT PERIODS:
   1. 0:29 to 0:31

👥 PLAYER TRACKING:
   Players tracked: 427

🎬 CLIP GENERATION:
   Impact clips: 21
   Normal clips: 177
   Failed clips: 0
   Total successful: 198
   Success rate: 100.0%
   Impact ratio: 10.6%

📁 OUTPUT DIRECTORY: Mitch Owens

✅ Processing completed successfully!

🧹 Memory cleanup completed.
